<a href="https://colab.research.google.com/github/rhf24/week9/blob/main/Week9_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.87MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!pip install langchain_huggingface langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
import pandas as pd
import os
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq # Need API Key
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
df = pd.read_csv('Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
import os
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [ ]:
# Loop through the dataset and generate markdown files
for i in range(0,len(df)):
    violation = df['المخالفة'].iloc[i]
    fine = df['الغرامة'].iloc[i]

    # Construct the markdown content
    markdown_content = f"# Violation {i+1}: {violation}\n\n"
    markdown_content += f"**Fine:** {fine}\n\n"

    # Write to a markdown file
    with open(f'{directory}/{i+1}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

print(f"Markdown files created for {len(df)} violations.")

Markdown files created for 104 violations.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,)

In [ ]:
import markdown
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Load the CAMeL-Lab model for embedding generation
embedding_function = SentenceTransformerEmbeddings(model_name="CAMeL-Lab/bert-base-arabic-camelbert-da")

# Create Chroma vector store and persist the embeddings
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db_camelbert")

print(f"Embeddings generated and stored in Chroma for {len(documents)} document chunks.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings generated and stored in Chroma for 104 document chunks.


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [ ]:
from langchain import PromptTemplate

PROMPT_TEMPLATE="""سيتم تزويدك بسياق يتضمن معلومات عن المخالفات المرورية والغرامات. و يجب عليك الإجابة على الأسئلة بناءً على السياق المتوفر، يرجى تقديم إجابات شاملة و واضحة باللغة العربية
المخالفة: {المخالفة}
الغرامة: {الغرامة}
الإجابة:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["الغرامة", "المخالفة"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
#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

In [ ]:
!pip install groq

In [ ]:
from langchain_groq import ChatGroq
groq_api_key = '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'
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
from langchain.chains import LLMChain
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"الغرامة": context_text, "المخالفة": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:
سيتم تزويدك بسياق يتضمن معلومات عن المخالفات المرورية والغرامات. و يجب عليك الإجابة على الأسئلة بناءً على السياق المتوفر، يرجى تقديم إجابات شاملة و واضحة باللغة العربية
المخالفة: ماهي الغرامة على القيادة بدون رخصة؟
الغرامة: <h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 46: عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم و

{'الغرامة': '<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>\n<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>\n\n<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>\n<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>\n\n<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>\n<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>\n\n<h1>Violation 46: عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>\n<p><strong>Fine:</strong> الغرامة المالية 500 - 900 ريال</p>',
 'المخالفة': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'الغرامة على القيادة بدون رخصة هي الغرامة المالية 150 - 300 ريال.'}

In [ ]:
print(f'Context:\n{response["الغرامة"]}\n\nQuestion:\n{response["المخالفة"]}\n\nText: \n{response["text"]}')

Context:
<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 10: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.</h1>
<p><strong>Fine:</strong> الغرامة المالية 150 - 300 ريال</p>

<h1>Violation 46: عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك.</h1>
<p><strong>Fine:</strong> الغرامة المالية 500 - 900 ريال</p>

Question:
ماهي الغرامة على القيادة بدون رخصة؟

Text: 
الغرامة على القيادة بدون رخصة هي الغرامة المالية 150 - 300 ريال.


- Raniyah Alghamdi
- Rahaf Alzahrani
- Maryam Jathmo